# ReAct Agent Implementation (Lesson 14)

**Objective:** Build a ReAct (Reasoning + Acting) agent that dynamically plans and executes actions based on observations.

**Learning Goals:**
- Understand the Thought-Action-Observation loop
- Implement tool selection and execution
- Handle errors and iterative refinement
- Track agent performance metrics

**Prerequisites:**
- Lesson 10 (AI-as-Judge) for LLM prompting patterns
- `backend/agent_evaluation.py` for validation functions
- `lesson-14/react_reflexion_patterns.md` for theoretical background

**Execution Modes:**
- **DEMO mode**: 3 simple tasks, <$0.50, ~3 min execution
- **FULL mode**: 15 complex tasks, <$3, ~10 min execution

---

## Setup and Configuration

In [5]:
# Cell 1: Imports and setup
import json
import time
from dataclasses import dataclass, field
from pathlib import Path
from typing import Any

import litellm
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Execution mode: "DEMO" (cheap, fast) or "FULL" (comprehensive)
MODE = "DEMO"  # Change to "FULL" for complete evaluation

# Configuration
CONFIG = {
    "DEMO": {
        "num_tasks": 3,
        "max_steps": 5,
        "model": "gpt-4o-mini",
        "estimated_cost": "$0.30-0.50",
        "estimated_time": "3-5 minutes"
    },
    "FULL": {
        "num_tasks": 15,
        "max_steps": 10,
        "model": "gpt-4o-mini",
        "estimated_cost": "$2-3",
        "estimated_time": "8-12 minutes"
    }
}

config = CONFIG[MODE]
print(f"🔧 Mode: {MODE}")
print(f"📊 Tasks: {config['num_tasks']}")
print(f"💰 Est. Cost: {config['estimated_cost']}")
print(f"⏱️  Est. Time: {config['estimated_time']}")
print(f"🤖 Model: {config['model']}")

🔧 Mode: DEMO
📊 Tasks: 3
💰 Est. Cost: $0.30-0.50
⏱️  Est. Time: 3-5 minutes
🤖 Model: gpt-4o-mini


## Tool Definitions

Define available tools for the ReAct agent to use.

In [1]:
# Cell 2: Tool definitions

# Mock recipe database
RECIPE_DB = [
    {"id": 1, "name": "Vegan Pasta Primavera", "cuisine": "Italian", "diet": "vegan", "time": 30, "ingredients":
        ["pasta", "vegetables", "olive oil"]},
    {"id": 2, "name": "Chicken Tikka Masala", "cuisine": "Indian", "diet": "none", "time": 45, "ingredients":
        ["chicken", "yogurt", "spices"]},
    {"id": 3, "name": "Gluten-Free Pizza", "cuisine": "Italian", "diet": "gluten-free", "time": 25, "ingredients": 
        ["gf flour", "cheese", "tomato"]},
    {"id": 4, "name": "Thai Green Curry", "cuisine": "Thai", "diet": "none", "time": 35, "ingredients": 
        ["curry paste", "coconut milk", "vegetables"]},
    {"id": 5, "name": "Keto Avocado Salad", "cuisine": "American", "diet": "keto", "time": 15, "ingredients":
        ["avocado", "eggs", "bacon"]}
]

SHOPPING_LIST = []

def search_recipes(cuisine: str = None, dietary_restrictions: list[str] = None, max_cook_time: int = None) -> list[dict]:
    """Search recipe database with filters.
    
    Args:
        cuisine: Filter by cuisine type
        dietary_restrictions: Filter by diet (vegan, gluten-free, keto)
        max_cook_time: Maximum cooking time in minutes
    
    Returns:
        List of matching recipes
    """
    results = RECIPE_DB.copy()
    
    if cuisine:
        results = [r for r in results if r["cuisine"].lower() == cuisine.lower()]
    
    if dietary_restrictions:
        for diet in dietary_restrictions:
            results = [r for r in results if r["diet"] == diet]
    
    if max_cook_time:
        results = [r for r in results if r["time"] <= max_cook_time]
    
    return results

def get_recipe_details(recipe_id: int) -> dict:
    """Get full recipe details by ID.
    
    Args:
        recipe_id: Recipe ID
    
    Returns:
        Recipe details or error
    """
    for recipe in RECIPE_DB:
        if recipe["id"] == recipe_id:
            return recipe
    return {"error": f"Recipe {recipe_id} not found"}

def add_to_shopping_list(ingredients: list[str]) -> dict:
    """Add ingredients to shopping list.
    
    Args:
        ingredients: List of ingredients to add
    
    Returns:
        Success message with updated list
    """
    global SHOPPING_LIST
    SHOPPING_LIST.extend(ingredients)
    return {"success": True, "shopping_list": SHOPPING_LIST, "count": len(SHOPPING_LIST)}

# Tool registry
TOOLS = {
    "search_recipes": {
        "function": search_recipes,
        "description": "Search recipe database by cuisine, dietary restrictions, or cooking time",
        "parameters": {
            "cuisine": {"type": "str", "required": False, "description": "Cuisine type (Italian, Indian, Thai, etc.)"},
            "dietary_restrictions": {"type": "list[str]", "required": False, "description": "Diet filters (vegan, gluten-free, keto)"},
            "max_cook_time": {"type": "int", "required": False, "description": "Max cooking time in minutes"}
        }
    },
    "get_recipe_details": {
        "function": get_recipe_details,
        "description": "Get full recipe details by ID",
        "parameters": {
            "recipe_id": {"type": "int", "required": True, "description": "Recipe ID"}
        }
    },
    "add_to_shopping_list": {
        "function": add_to_shopping_list,
        "description": "Add ingredients to shopping list",
        "parameters": {
            "ingredients": {"type": "list[str]", "required": True, "description": "List of ingredients"}
        }
    }
}

print(f"✅ Loaded {len(TOOLS)} tools: {list(TOOLS.keys())}")
print(f"📦 Recipe database: {len(RECIPE_DB)} recipes")

✅ Loaded 3 tools: ['search_recipes', 'get_recipe_details', 'add_to_shopping_list']
📦 Recipe database: 5 recipes


## ReAct Agent Implementation

Implement the ReAct agent with Thought-Action-Observation loop.

In [6]:
# Cell 3: ReActAgent class

@dataclass
class ReActState:
    """State for ReAct agent execution."""
    query: str
    history: list[dict] = field(default_factory=list)
    observations: list[dict] = field(default_factory=list)
    data: dict = field(default_factory=dict)
    errors: list[str] = field(default_factory=list)
    step_count: int = 0
    max_steps: int = 10
    done: bool = False

class ReActAgent:
    """ReAct agent with Thought-Action-Observation loop."""
    
    def __init__(self, llm_model: str = "gpt-4o-mini", max_steps: int = 10, tools: dict = None):
        """Initialize ReAct agent.
        
        Args:
            llm_model: LLM model for reasoning
            max_steps: Maximum iterations before timeout
            tools: Available tools dictionary
        """
        self.llm_model = llm_model
        self.max_steps = max_steps
        self.tools = tools or {}
        self.total_tokens = 0
        self.total_cost = 0.0
    
    def _generate_thought(self, state: ReActState) -> str:
        """Generate reasoning thought based on current state.
        
        Args:
            state: Current agent state
        
        Returns:
            Thought string with reasoning
        """
        # Build context from history
        context = f"Query: {state.query}\n\n"
        
        if state.history:
            context += "Previous steps:\n"
            for i, entry in enumerate(state.history[-6:]):  # Last 6 entries
                context += f"{i+1}. {entry['type'].upper()}: {str(entry.get('content', entry))[:150]}\n"
        
        # Tool descriptions
        tool_desc = "\n".join([f"- {name}: {tool['description']}" for name, tool in self.tools.items()])
        
        prompt = f"""{context}

Available tools:
{tool_desc}

You are a ReAct agent. Generate your next thought following this format:

Thought: [Your reasoning about what to do next]
Action: [tool_name]
Action Input: {{"param": "value"}}

OR if you have the final answer:

Thought: I now have enough information to answer
Final Answer: [Your complete answer]

Generate your thought:"""
        
        try:
            response = litellm.completion(
                model=self.llm_model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.0
            )
            
            # Track costs
            usage = response.usage
            self.total_tokens += usage.total_tokens
            # Approximate cost: $0.15/1M input, $0.60/1M output for gpt-4o-mini
            self.total_cost += (usage.prompt_tokens * 0.15 / 1_000_000) + (usage.completion_tokens * 0.60 / 1_000_000)
            
            return response.choices[0].message.content
        except Exception as e:
            return f"Thought: Error generating thought: {str(e)}\nFinal Answer: Unable to complete task due to error."
    
    def _is_final_answer(self, thought: str) -> bool:
        """Check if thought contains final answer."""
        return "Final Answer:" in thought or "final answer" in thought.lower()
    
    def _extract_answer(self, thought: str) -> str:
        """Extract final answer from thought."""
        if "Final Answer:" in thought:
            return thought.split("Final Answer:")[1].strip()
        return thought
    
    def _parse_action(self, thought: str) -> tuple[str, dict]:
        """Parse action and arguments from thought.
        
        Returns:
            Tuple of (tool_name, arguments_dict)
        """
        try:
            # Extract action
            if "Action:" in thought:
                action_line = thought.split("Action:")[1].split("\n")[0].strip()
            else:
                return "search_recipes", {}  # Default fallback
            
            # Extract action input
            if "Action Input:" in thought:
                input_str = thought.split("Action Input:")[1].strip()
                # Try to parse JSON
                try:
                    args = json.loads(input_str.split("\n")[0])
                except json.JSONDecodeError:
                    args = {}
            else:
                args = {}
            
            return action_line, args
        except Exception:
            return "search_recipes", {}  # Safe fallback
    
    def _execute_action(self, thought: str, state: ReActState) -> dict:
        """Execute action from thought.
        
        Returns:
            Result dictionary with tool, args, observation, status
        """
        tool_name, args = self._parse_action(thought)
        
        # Validate tool exists
        if tool_name not in self.tools:
            return {
                "tool": tool_name,
                "args": args,
                "observation": f"Error: Tool '{tool_name}' not found. Available: {list(self.tools.keys())}",
                "status": "error"
            }
        
        # Execute tool
        try:
            tool = self.tools[tool_name]
            result = tool["function"](**args)
            
            # Format observation
            if isinstance(result, list):
                obs = f"Found {len(result)} results: {result}"
            elif isinstance(result, dict) and "error" in result:
                obs = f"Error: {result['error']}"
                return {
                    "tool": tool_name,
                    "args": args,
                    "observation": obs,
                    "status": "error"
                }
            else:
                obs = str(result)
            
            return {
                "tool": tool_name,
                "args": args,
                "observation": obs,
                "status": "success",
                "result": result
            }
        except Exception as e:
            return {
                "tool": tool_name,
                "args": args,
                "observation": f"Error executing tool: {str(e)}",
                "status": "error"
            }
    
    def run(self, query: str) -> dict[str, Any]:
        """Run ReAct loop until completion or max steps.
        
        Args:
            query: User query
        
        Returns:
            Result with trajectory, answer, and metrics
        """
        start_time = time.time()
        
        # Initialize state
        state = ReActState(query=query, max_steps=self.max_steps)
        
        # ReAct loop
        while not state.done and state.step_count < state.max_steps:
            # Phase 1: Generate Thought
            thought = self._generate_thought(state)
            state.history.append({"type": "thought", "content": thought, "step": state.step_count})
            
            # Check if final answer
            if self._is_final_answer(thought):
                answer = self._extract_answer(thought)
                state.done = True
                state.history.append({"type": "answer", "content": answer})
                break
            
            # Phase 2: Execute Action
            action_result = self._execute_action(thought, state)
            state.history.append({
                "type": "action",
                "tool": action_result["tool"],
                "args": action_result["args"],
                "step": state.step_count
            })
            
            # Phase 3: Process Observation
            observation = action_result["observation"]
            state.observations.append({
                "step": state.step_count,
                "observation": observation,
                "status": action_result["status"]
            })
            state.history.append({"type": "observation", "content": observation, "step": state.step_count})
            
            # Update state
            state.step_count += 1
            
            # Check for repeated errors
            if action_result["status"] == "error":
                state.errors.append(observation)
                if len(state.errors) >= 3:
                    state.done = True
                    state.history.append({
                        "type": "answer",
                        "content": f"Failed to complete task after {len(state.errors)} errors"
                    })
                    break
        
        # Timeout check
        if not state.done:
            state.history.append({
                "type": "answer",
                "content": f"Max steps ({state.max_steps}) reached without completion"
            })
        
        execution_time = time.time() - start_time
        
        # Extract final answer
        final_answer = "No answer generated"
        for entry in reversed(state.history):
            if entry["type"] == "answer":
                final_answer = entry["content"]
                break
        
        return {
            "query": query,
            "answer": final_answer,
            "trajectory": state.history,
            "observations": state.observations,
            "metrics": {
                "steps": state.step_count,
                "completed": state.done,
                "errors": len(state.errors),
                "execution_time": execution_time,
                "total_tokens": self.total_tokens,
                "total_cost": self.total_cost
            }
        }

print("✅ ReActAgent class defined")

✅ ReActAgent class defined


## Test Tasks

Define test tasks for DEMO and FULL modes.

In [7]:
# Cell 4: Test tasks

DEMO_TASKS = [
    "Find vegan Italian recipes",
    "Get details for recipe ID 2",
    "Find quick recipes under 20 minutes"
]

FULL_TASKS = DEMO_TASKS + [
    "Find gluten-free Italian recipes and add their ingredients to shopping list",
    "Search for Thai cuisine and get details of the first result",
    "Find keto recipes with cooking time under 30 minutes",
    "Get recipe 4 details and add its ingredients to shopping list",
    "Find all Indian recipes",
    "Search for vegan recipes under 25 minutes",
    "Find recipes without dietary restrictions",
    "Get details for recipe 5 and analyze ingredients",
    "Find Italian recipes under 30 minutes",
    "Search for recipes with maximum 40 minute cooking time",
    "Find all available cuisines and recommend one",
    "Create a complete meal plan with shopping list"
]

tasks = DEMO_TASKS if MODE == "DEMO" else FULL_TASKS

print(f"📋 Loaded {len(tasks)} test tasks for {MODE} mode")
print("\nSample tasks:")
for i, task in enumerate(tasks[:3], 1):
    print(f"  {i}. {task}")

📋 Loaded 3 test tasks for DEMO mode

Sample tasks:
  1. Find vegan Italian recipes
  2. Get details for recipe ID 2
  3. Find quick recipes under 20 minutes


## Execute ReAct Agent

Run the agent on all test tasks and collect results.

In [8]:
# Cell 5: Execute agent

print(f"🚀 Starting ReAct agent execution ({MODE} mode)...\n")
print(f"⚠️  Estimated cost: {config['estimated_cost']}")
print(f"⏱️  Estimated time: {config['estimated_time']}\n")

# Initialize agent
agent = ReActAgent(
    llm_model=config["model"],
    max_steps=config["max_steps"],
    tools=TOOLS
)

results = []
start_time = time.time()

for i, task in enumerate(tasks, 1):
    print(f"\n{'='*80}")
    print(f"Task {i}/{len(tasks)}: {task}")
    print(f"{'='*80}")
    
    try:
        # Reset shopping list for each task
        SHOPPING_LIST.clear()
        
        # Run agent
        result = agent.run(task)
        results.append(result)
        
        # Display summary
        print("\n📊 Result Summary:")
        print(f"   Steps: {result['metrics']['steps']}")
        print(f"   Completed: {result['metrics']['completed']}")
        print(f"   Errors: {result['metrics']['errors']}")
        print(f"   Time: {result['metrics']['execution_time']:.2f}s")
        print(f"   Cost: ${result['metrics']['total_cost']:.4f}")
        print(f"\n💬 Final Answer:\n   {result['answer'][:200]}...")
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        results.append({
            "query": task,
            "answer": f"Error: {str(e)}",
            "trajectory": [],
            "metrics": {"steps": 0, "completed": False, "errors": 1, "execution_time": 0, "total_tokens": 0, "total_cost": 0}
        })

total_time = time.time() - start_time

print(f"\n\n{'='*80}")
print("✅ Execution Complete")
print(f"{'='*80}")
print(f"Total tasks: {len(results)}")
print(f"Total time: {total_time:.2f}s ({total_time/60:.1f} min)")
print(f"Total cost: ${agent.total_cost:.4f}")
print(f"Total tokens: {agent.total_tokens:,}")

🚀 Starting ReAct agent execution (DEMO mode)...

⚠️  Estimated cost: $0.30-0.50
⏱️  Estimated time: 3-5 minutes


Task 1/3: Find vegan Italian recipes

📊 Result Summary:
   Steps: 4
   Completed: True
   Errors: 1
   Time: 17.16s
   Cost: $0.0005

💬 Final Answer:
   Here are some popular vegan Italian recipes you can try:

1. **Vegan Pasta Primavera**: A mix of seasonal vegetables sautéed and tossed with pasta and olive oil.
2. **Vegan Risotto**: Creamy risotto m...

Task 2/3: Get details for recipe ID 2

📊 Result Summary:
   Steps: 4
   Completed: True
   Errors: 3
   Time: 14.13s
   Cost: $0.0009

💬 Final Answer:
   Failed to complete task after 3 errors...

Task 3/3: Find quick recipes under 20 minutes

📊 Result Summary:
   Steps: 5
   Completed: False
   Errors: 2
   Time: 10.23s
   Cost: $0.0013

💬 Final Answer:
   Max steps (5) reached without completion...


✅ Execution Complete
Total tasks: 3
Total time: 41.52s (0.7 min)
Total cost: $0.0013
Total tokens: 5,050


## Analyze Results

Calculate performance metrics and analyze agent behavior.

In [9]:
# Cell 6: Analyze results

print("📊 ReAct Agent Performance Analysis\n")

# Calculate metrics
total_tasks = len(results)
completed_tasks = sum(1 for r in results if r["metrics"]["completed"])
total_steps = sum(r["metrics"]["steps"] for r in results)
total_errors = sum(r["metrics"]["errors"] for r in results)
avg_steps = total_steps / total_tasks if total_tasks > 0 else 0
avg_time = sum(r["metrics"]["execution_time"] for r in results) / total_tasks if total_tasks > 0 else 0

completion_rate = completed_tasks / total_tasks if total_tasks > 0 else 0
error_rate = total_errors / total_steps if total_steps > 0 else 0

print("Completion Metrics:")
print(f"  ✅ Completion rate: {completion_rate:.1%} ({completed_tasks}/{total_tasks})")
print(f"  📈 Average steps per task: {avg_steps:.1f}")
print(f"  ⏱️  Average time per task: {avg_time:.2f}s")
print(f"  ❌ Error rate: {error_rate:.1%} ({total_errors} errors in {total_steps} steps)")

print("\nCost Metrics:")
print(f"  💰 Total cost: ${agent.total_cost:.4f}")
print(f"  💵 Cost per task: ${agent.total_cost/total_tasks:.4f}")
print(f"  🔢 Total tokens: {agent.total_tokens:,}")
print(f"  📊 Tokens per task: {agent.total_tokens//total_tasks:,}")

# Tool usage analysis
tool_usage = {}
for result in results:
    for entry in result["trajectory"]:
        if entry["type"] == "action":
            tool = entry["tool"]
            tool_usage[tool] = tool_usage.get(tool, 0) + 1

print("\nTool Usage:")
for tool, count in sorted(tool_usage.items(), key=lambda x: x[1], reverse=True):
    print(f"  {tool}: {count} calls ({count/total_steps*100:.1f}% of actions)")

# Success analysis
successful = [r for r in results if r["metrics"]["completed"] and r["metrics"]["errors"] == 0]
failed = [r for r in results if not r["metrics"]["completed"] or r["metrics"]["errors"] > 0]

print("\nTask Categories:")
print(f"  ✅ Successful (no errors): {len(successful)} ({len(successful)/total_tasks*100:.1f}%)")
print(f"  ⚠️  Failed or with errors: {len(failed)} ({len(failed)/total_tasks*100:.1f}%)")

if failed:
    print("\nFailed Tasks:")
    for r in failed[:3]:  # Show first 3
        print(f"  - {r['query'][:60]}... (errors: {r['metrics']['errors']})")

📊 ReAct Agent Performance Analysis

Completion Metrics:
  ✅ Completion rate: 66.7% (2/3)
  📈 Average steps per task: 4.3
  ⏱️  Average time per task: 13.84s
  ❌ Error rate: 46.2% (6 errors in 13 steps)

Cost Metrics:
  💰 Total cost: $0.0013
  💵 Cost per task: $0.0004
  🔢 Total tokens: 5,050
  📊 Tokens per task: 1,683

Tool Usage:
  search_recipes: 10 calls (76.9% of actions)
  get_recipe_details: 2 calls (15.4% of actions)
  None: 1 calls (7.7% of actions)

Task Categories:
  ✅ Successful (no errors): 0 (0.0%)
  ⚠️  Failed or with errors: 3 (100.0%)

Failed Tasks:
  - Find vegan Italian recipes... (errors: 1)
  - Get details for recipe ID 2... (errors: 3)
  - Find quick recipes under 20 minutes... (errors: 2)


## Save Results

Save results to JSON for dashboard integration.

In [10]:
# Cell 7: Save results

output_dir = Path("lesson-14/results")
output_dir.mkdir(parents=True, exist_ok=True)

output_data = {
    "metadata": {
        "mode": MODE,
        "model": config["model"],
        "num_tasks": len(tasks),
        "max_steps": config["max_steps"],
        "execution_date": time.strftime("%Y-%m-%d %H:%M:%S"),
        "total_time": total_time,
        "total_cost": agent.total_cost,
        "total_tokens": agent.total_tokens
    },
    "summary": {
        "completion_rate": completion_rate,
        "avg_steps_per_task": avg_steps,
        "avg_time_per_task": avg_time,
        "error_rate": error_rate,
        "successful_tasks": len(successful),
        "failed_tasks": len(failed),
        "tool_usage": tool_usage
    },
    "results": results
}

output_path = output_dir / f"react_agent_results_{MODE.lower()}.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"✅ Results saved to: {output_path}")
print(f"📁 File size: {output_path.stat().st_size / 1024:.1f} KB")

# Also save a general planning_validation.json for dashboard
dashboard_data = {
    "version": "1.0",
    "created": time.strftime("%Y-%m-%d"),
    "mode": MODE,
    "metrics": {
        "planning_accuracy": completion_rate,
        "avg_steps": avg_steps,
        "error_rate": error_rate,
        "completion_rate": completion_rate
    },
    "tool_usage": tool_usage,
    "sample_trajectories": [
        {
            "query": r["query"],
            "steps": r["metrics"]["steps"],
            "completed": r["metrics"]["completed"],
            "answer": r["answer"][:200]
        }
        for r in results[:5]
    ]
}

dashboard_path = output_dir / "planning_validation.json"
with open(dashboard_path, "w", encoding="utf-8") as f:
    json.dump(dashboard_data, f, indent=2, ensure_ascii=False)

print(f"✅ Dashboard data saved to: {dashboard_path}")
print("\n🎉 Notebook execution complete!")

✅ Results saved to: lesson-14/results/react_agent_results_demo.json
📁 File size: 18.6 KB
✅ Dashboard data saved to: lesson-14/results/planning_validation.json

🎉 Notebook execution complete!


## Validation and Assertions

Verify results meet quality thresholds.

In [11]:
# Cell 8: Validation

print("🔍 Validating results...\n")

# Validation checks
checks = [
    ("All tasks executed", len(results) == len(tasks)),
    ("Completion rate ≥50%", completion_rate >= 0.5),
    ("Average steps ≤ max_steps", avg_steps <= config["max_steps"]),
    ("Cost within budget", agent.total_cost <= float(config["estimated_cost"].split("-")[1].replace("$", ""))),
    ("Execution time reasonable", total_time <= int(config["estimated_time"].split("-")[1].split()[0]) * 60),
    ("At least one tool used", len(tool_usage) > 0),
    ("No task had 0 steps", all(r["metrics"]["steps"] > 0 or r["metrics"]["errors"] > 0 for r in results))
]

passed = 0
for check_name, check_result in checks:
    status = "✅" if check_result else "❌"
    print(f"{status} {check_name}")
    if check_result:
        passed += 1

print(f"\n📊 Validation: {passed}/{len(checks)} checks passed ({passed/len(checks)*100:.1f}%)")

if passed == len(checks):
    print("\n🎉 All validation checks passed!")
elif passed >= len(checks) * 0.8:
    print("\n⚠️  Most checks passed, but some issues detected")
else:
    print("\n❌ Multiple validation failures - review results")

print("\n" + "="*80)
print("Notebook execution complete. Results saved to lesson-14/results/")
print("="*80)

🔍 Validating results...

✅ All tasks executed
✅ Completion rate ≥50%
✅ Average steps ≤ max_steps
✅ Cost within budget
✅ Execution time reasonable
✅ At least one tool used
✅ No task had 0 steps

📊 Validation: 7/7 checks passed (100.0%)

🎉 All validation checks passed!

Notebook execution complete. Results saved to lesson-14/results/
